# Single Module CPU Trigger Test

# Test Description
This script will test the CPU triggering capability of proteus. In this different channels of the proteus module will get trigger when command *TRG executed from script. So It is totally software trigger. This test will run for multiple sampling rate from 1GHz to 2.5GHz for all channels and from 2.5GHz to 9GHz for Channel 1 & 3 only.

# Test Equipment
1. Tabor Proteus 9484M
2. Keysight Oscilloscope MSO9254A
3. 4 SMA to BNC Cable

# Hardware Connection

    Proteus Module    --->     Oscilloscope
    
    Channel 1         --->      Channel 1
    Channel 2         --->      Channel 2
    Channel 3         --->      Channel 3
    Channel 4         --->      Channel 4


# Test Procedure
1. Connect all the channels of Proteus module with the concerned channel of oscilloscope.
2. Keep the oscilloscope on default.
3. Run the script and observe output on scope.

Proteus release- Aurora

In [1]:
# Import required Libraries
import os
import sys
import tempfile
import webbrowser
srcpath = os.path.realpath('../SourceFiles')
sys.path.append(srcpath)
from teproteus import TEProteusAdmin as TepAdmin
from teproteus import TEProteusInst as TepInst
from teproteus_functions_v3 import connect
from teproteus_functions_v3 import disconnect
from teproteus_functions_v3 import set_lib_dir_path
from teproteus_functions_v3 import get_cpatured_header
from teproteus_functions_v3 import gauss_env
from teproteus_functions_v3 import iq_kernel
from teproteus_functions_v3 import pack_kernel_data
from teproteus import TEProteusAdmin, TEProteusInst
from tevisainst import TEVisaInst
#matplotlib notebook
import pyvisa as visa
import numpy as np
import time
import ipywidgets as widgets
from IPython.core.debugger import set_trace
from scipy.signal import chirp, sweep_poly
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from scipy import signal
import math
import pdb

In [2]:
## Connect to instrument
connection = 'LAN' #input ("Do you wish to connect via LAN/PXI/LOCAL/USB")
if connection == 'PXI':   
    # not setting this command means DLL are taken from SYS32
    #set_lib_dir_path(r'D:\Projects\ProteusAwg_Anabelle\x64\Debug') 
    # for service connection enter instrument IP
    # for PXI DLL connection enter the module slot number with Auto=False
    # when Auto=True the lowest module will be connected
    proteus=connect("6",Auto=False)
    
elif connection == 'LAN':
    proteus_addr = 'TCPIP::192.90.70.22::5025::SOCKET'
    try:
        proteus = TEVisaInst(proteus_addr)
    except TEVisaInst.Error as  ex1:
        print('Couldn\'t connect to \'%s\', exiting now...' % proteus_addr)
        sys.exit()
        
elif connection == 'USB':
    proteus_addr = 'USB0::0x2A8D::0x900E::MY55490134::INSTR'
    try:
        proteus = TEVisaInst(proteus_addr)
    except TEVisaInst.Error as  ex1:
        print('Couldn\'t connect to \'%s\', exiting now...' % proteus_addr)
        sys.exit()
        
elif connection == 'LOCAL':
    proteus_addr = 'TCPIP::127.0.0.1::5025::SOCKET'
    try:
        proteus = TEVisaInst(proteus_addr)
    except TEVisaInst.Error as  ex1:
        print('Couldn\'t connect to \'%s\', exiting now...' % proteus_addr)
        sys.exit()
        
else:
    print("Please select the conection method first")
# Get the instrument's *IDN
resp = proteus.send_scpi_query('*IDN?')
print('Connected to: ' + resp)
proteus.default_paranoia_level = 2

Connected to: Tabor Electronics,P9484M,000002232771,1.238.6 --Tabor Electronics,P9484M,000000000000,1.238.6 --slot#: 2, slot#: 4, 


In [3]:
# Get the model:
model_name = proteus.send_scpi_query('SYST:INF:MODel?')
print('Model: {0} '.format(model_name))

# Get model dependant parameters:
if model_name.startswith('P948'):
    bpp = 2
    max_dac = 65535
    wpt_type = np.uint16
    offset_factor = 1
elif model_name.startswith('P908'):
    bpp = 1
    max_dac = 255
    wpt_type = np.uint8
    offset_factor = 1
else:
    bpp = 2
    max_dac = 65535
    wpt_type = np.uint16
    offset_factor = 2
    
half_dac = max_dac / 2.0    
# Get the DAC mode (8 bits or 16 bits)

resp = proteus.send_scpi_query(':SYST:INF:DAC?')
if resp == 'M0': 
    dac_mode=16 
else: dac_mode=8
   
print('DAC {0} bits'.format(dac_mode))

# Get number of channels
resp = proteus.send_scpi_query(":INST:CHAN? MAX")
print("Number of channels: " + resp)
num_channels = int(resp)

Model: P9484M 
DAC 16 bits
Number of channels: 4


In [4]:
# Several initializations ..
proteus.send_scpi_cmd('*CLS; *RST')
#proteus.send_scpi_cmd(':INST:CHAN 2')
proteus.send_scpi_cmd(':TRAC:DEL:ALL') # Delete all segments of the programmable channel's DDR.
resp = proteus.send_scpi_query(':SYST:ERR?')
print(resp)
# Get number of channels
resp = proteus.send_scpi_query(":INST:CHAN? MAX")
print("Number of channels: " + resp)
num_channels = int(resp)

0, no error
Number of channels: 4


In [5]:
scope_addr2=  'USB0::0x2A8D::0x900E::MY55490134::INSTR'

# connect to scope via USB
try:
    resourceManager = visa.ResourceManager()   # Create a connection (session) to the instrument
    scope = resourceManager.open_resource(scope_addr2)
except visa.Error as ex2:
        print('Couldn\'t connect to \'%s\', exiting now...' % scope_addr2)
        sys.exit()
    
    
    
## scope acquisition 
# Send *IDN? and read the response
scope.write('*RST?')
scope.write('*IDN?')
idn = scope.read()
print('*IDN? returned: %s' % idn.rstrip('\n'))






*IDN? returned: KEYSIGHT TECHNOLOGIES,MSO9254A,MY55490134,06.40.01101


In [6]:
InitialFreqSampleRate = 1000
MaximumFreqSampleRate = 9000
FreqSampleRateJump = 1000

# print( 'Internal trigger Test for sampling rate {}MHz to {}MHz in jumps of {}MHz'.format(InitialFreqSampleRate,MaximumFreqSampleRate,FreqSampleRateJump)) 
for FreqSampleRate in range (InitialFreqSampleRate,MaximumFreqSampleRate+FreqSampleRateJump,FreqSampleRateJump):
    proteus.send_scpi_cmd(':FREQ:RAST {} MHz'.format(int(FreqSampleRate)))
    FreqSampleRate = int(FreqSampleRate)
    
    print('***Test begin for sampling rate {}MHz ***\n'.format(FreqSampleRate))
        
    if FreqSampleRate > 2500:
        channel_select = [1,3]
    else:
        channel_select = [1,2,3,4]
    
    for in_channelselect in range(len(channel_select)):
        # Select Channel & Clear it for new operation
        proteus.send_scpi_cmd(':INST:CHAN {}'.format(channel_select[in_channelselect]))
        # Several initializations ..
        proteus.send_scpi_cmd('*CLS; *RST')
        #proteus.send_scpi_cmd(':INST:CHAN 2')
        proteus.send_scpi_cmd(':TRAC:DEL:ALL') # Delete all segments of the programmable channel's DDR.
#         resp = proteus.send_scpi_query(':SYST:ERR?')
#         print(resp)


        # Build waveforms on selected channel
        seglen = 1024
        ncycles = 1
        max_dac = 65535
        waves = [ None for _ in range(num_channels)]
        #Create Square wave
        for i in range(num_channels):
            cyclelen = seglen / ncycles

            x = np.linspace(start=0, stop=2 * np.pi * ncycles, num=seglen, endpoint=False)
            y = (np.sin(x)+1.0)*half_dac
            #y = (y <= cyclelen / 2) * max_dac
            y = np.round(y)
            y = np.clip(y, 0, max_dac)
            if dac_mode == 16:
                waves[i] = y.astype(np.uint16)
            else:
                waves[i] = y.astype(np.uint8)
#         plt.plot(x,y)
#         print("************* WAVEFORM*****************")
        # Download segments
        for ii in range(1):
            ichan = ii
            channb = ichan + channel_select[in_channelselect]
            segnum = 1
            wav = waves[0]
#             print('Download wave to segment {} of channel {}'.format(segnum, channb))
            print('\t-->Test Start for sampling rate {}MHz on channel {} <--'.format(FreqSampleRate,channel_select[in_channelselect]))
            # Select channel
            proteus.send_scpi_cmd(':INST:CHAN {}'.format(channb))
            # Define segment
            proteus.send_scpi_cmd(':TRAC:DEF {},{}'.format(segnum, seglen))
            # Select the segment
            proteus.send_scpi_cmd(':TRAC:SEL {}'.format(segnum))
            # Increase the timeout before writing binary-data:
            proteus.timeout = 30000
            # Select the segment
            cmd = ':TRAC:SEL {0}'.format(segnum)
            proteus.send_scpi_cmd(cmd)
            # Send the binary-data:
            proteus.write_binary_data(':TRAC:DATA', wav)
            resp = proteus.send_scpi_query(':SYST:ERR?')
            resp = resp.rstrip()
            if not resp.startswith('0'):
                print('ERROR: "{}" after writing binary values'.format(resp))
            # Play the specified segment at the selected channel:
            cmd = ':SOUR:FUNC:MODE:SEGM {}'.format(segnum)
            proteus.send_scpi_cmd(cmd)
            # Turn on the output of the selected channel:
            proteus.send_scpi_cmd(':OUTP ON')
            resp = proteus.send_scpi_query(':SYST:ERR?')
#             print(resp)
#         print()
#         resp = proteus.send_scpi_query(':SYST:ERR?')
#         print(resp)
        
        
        # Read peak_peak_volt for sine wave generated
        scope.write('*CLS;:DISPlay:CGRade:LEVels ')
        scope.write('AUTOscale')
        time.sleep(10)
        scope.write(':MEASure:VPP CHANnel{}'.format(channel_select[in_channelselect]))
        time.sleep(2)
        scope.write(':MEASure:RESults?')
        result = scope.read()
        peak_peak_volt1= float(result.split(',')[1])
#         print(peak_peak_volt1)
        
        
        
        # Make contineous mode OFF to turn trigger mode ON
        # Read peak_peak_volt again, expected voltage level is around zero as contineous mode is off
        proteus.send_scpi_cmd(':INIT:CONT OFF')
        scope.write(':CHANnel{}:DISPlay ON'.format(channel_select[in_channelselect]))
        proteus.send_scpi_cmd(':OUTP ON')
        # Get the status of contineous mode 1= ON, 0=OFF
        resp = proteus.send_scpi_query(":INIT:CONT?")
#         print("State of contineous mode is : " + resp)
        scope.write('*CLS;:DISPlay:CGRade:LEVels ')
#       scope.write('AUTOscale')
        time.sleep(5)    
        scope.write(':MEASure:VPP CHANnel{}'.format(channel_select[in_channelselect]))
        time.sleep(2)
        scope.write(':MEASure:RESults?')
        result = scope.read()
        peak_peak_volt2= float(result.split(',')[1])
#         print(peak_peak_volt2)
    
        
        
        # Enaable the Trigger source NONE|TRG1|TRG2|TRG3|TRG4|TRG5|TRG6|INTernal|CPU|FBTRg|HWControl
        proteus.send_scpi_cmd(':TRIG:SOUR:ENAB CPU')
        # Enable the triggering source
        resp = proteus.send_scpi_query(":TRIG:SOUR:ENAB?")
#         print("Triggering source " + resp + " is Enabled")
        # Activate the triggering source 1= ON, 0=OFF
        proteus.send_scpi_cmd(':TRIG:STAT ON')
        
        
        # After it appear adjust the Verticle and Horizontal scaling so that voltage level can be read
        scope.write(':CHANnel{}:PROBe 1.0'.format(channel_select[in_channelselect]))
        scope.write(':CHANnel{}:SCALe 100E-3'.format(channel_select[in_channelselect]))
        scope.write(':CHANnel{}:OFFSet 0.0'.format(channel_select[in_channelselect]))
        scope.write(':TIMebase:SCALe 5E-6')
        scope.write(':TRIGger:SWEep TRIGgered')
        scope.write(':TRIGger:MODE EDGE')
        scope.write(':TRIGger:EDGE:SOURce CHANnel{}'.format(channel_select[in_channelselect]))
        scope.write(':TRIGger:LEVel CHANnel{},120e-3'.format(channel_select[in_channelselect]))
        time.sleep(2)
        
        proteus.send_scpi_cmd('*TRG')
        
        time.sleep(2)
        scope.write(':MEASure:PERiod? CHANnel{}'.format(channel_select[in_channelselect]))
        resualt = scope.read()
        measured_period = float(resualt.split(',')[0])
#         print(measured_period)
        scope.write(':MEASure:VPP CHANnel{}'.format(channel_select[in_channelselect]))
        time.sleep(2)
        scope.write(':MEASure:RESults?')
        result = scope.read()
        peak_peak_volt3= float(result.split(',')[1])
#         print(peak_peak_volt3)
        
        
        
        # Test for Verticle and Horizontal parameters for decision that was captured on different level
        if abs(peak_peak_volt1-peak_peak_volt3)<120e-3 and peak_peak_volt2< 120e-03:
            print('\tTest Pass for sampling rate {0}MHz for channel {1}\n'.format(FreqSampleRate,channel_select[in_channelselect]))
        else:
            print('\tTest Fail for sampling rate {0}MHz for channel {1}\n'.format(FreqSampleRate,channel_select[in_channelselect]))
      
        # Clear the display for next operation
        for i in range(4):
            scope.write(':CHANnel{}:DISPlay OFF'.format(i+1))
            proteus.send_scpi_cmd(':INST:CHAN {}'.format(channel_select[in_channelselect]))
            proteus.send_scpi_cmd(':TRIG:STAT OFF')
            
     
    print('Test completed for sampling rate {}MHz\n'.format(FreqSampleRate))
            
           
print('***Test Completed for all the sampling rates***')
# close proteus connection
proteus.close_instrument()
disconnect()

# Close the connection to the instrument
scope.close()
resourceManager.close()

***Test begin for sampling rate 1000MHz ***

	-->Test Start for sampling rate 1000MHz on channel 1 <--
	Test Pass for sampling rate 1000MHz for channel 1

	-->Test Start for sampling rate 1000MHz on channel 2 <--
	Test Pass for sampling rate 1000MHz for channel 2

	-->Test Start for sampling rate 1000MHz on channel 3 <--
	Test Pass for sampling rate 1000MHz for channel 3

	-->Test Start for sampling rate 1000MHz on channel 4 <--
	Test Pass for sampling rate 1000MHz for channel 4

Test completed for sampling rate 1000MHz

***Test begin for sampling rate 2000MHz ***

	-->Test Start for sampling rate 2000MHz on channel 1 <--
	Test Pass for sampling rate 2000MHz for channel 1

	-->Test Start for sampling rate 2000MHz on channel 2 <--
	Test Pass for sampling rate 2000MHz for channel 2

	-->Test Start for sampling rate 2000MHz on channel 3 <--
	Test Pass for sampling rate 2000MHz for channel 3

	-->Test Start for sampling rate 2000MHz on channel 4 <--
	Test Pass for sampling rate 2000MHz for

In [7]:
disconnect()
scope.close()
print('Disconnected')

Disconnected
